In [1]:
import cv2
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [2]:
model = load_model('./Emb_Recogizer.h5')
FRmodel = load_model('./face-rec_Google.h5')
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')

/usr/local/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
names = {
    0 : 'shivang',
    1 : 'suransh',
    2 : 'tanishq'
}

In [4]:
shivang = np.load('./data/shivang.npy')
suri = np.load('./data/suransh.npy')
print(shivang.shape)
print(suri.shape)

(128,)
(128,)


In [5]:
def predictSimple(cur):
    fin_img = cv2.resize(cur, (100,100) )
    fin_img = np.array(fin_img)
    emb = img_to_embedding(fin_img, FRmodel)
    dist1 = np.linalg.norm(emb-shivang)
    dist2 = np.linalg.norm(emb-shivang)
    if dist1<dist2:
        return 0
    else:
        return 1

In [6]:
def img_to_embedding(image, model):
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict(x_train)
    return embedding

In [7]:
def predict(cur):
    fin_img = cv2.resize(cur, (100,100) )
    fin_img = np.array(fin_img)
    emb = img_to_embedding(fin_img, FRmodel)
    emb = np.reshape(emb,(1,128,))
    result = model.predict( emb )
    num = np.argmax( result[0] )
    conf = np.max(result[0])
    return conf,num

In [9]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if ret==False:
        continue
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    for face in faces[:1]:
        x,y,w,h = face
        offset = 7
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
#         out = predict(face_section)
        conf,out = predict(face_section)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,255), 2)
        cv2.putText(frame, names[int(out)]+"("+str(conf)+")",(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow("Faces", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

-1